In [80]:
import numpy as np
import pygame
from copy import deepcopy
import random
import pandas as pd
import datetime

FPS = 20

MINIMAX_DEPTH = 1

WIN_WIDTH = 800
WIN_HEIGHT = 800


BOARD_WIDTH = 8

FIELD_SIZE = WIN_WIDTH/BOARD_WIDTH
PIECE_SIZE = FIELD_SIZE/2 - 8
MARK_THICK = 2
POS_MOVE_MARK_SIZE = PIECE_SIZE/2


WHITE = (255,255,255)
BLACK = (0,0,0)
BLUE = (0,0,255)
RED = (255,0,0)
GREEN = (0,255,0)

class Move:
    def __init__(self, piece, dest_row, dest_col, captures=None):
        self.piece=piece
        self.dest_row=dest_row
        self.dest_col=dest_col
        self.captures=captures

class Field:
    def draw(self):
        pass
    
    def is_empty(self):
        return True
    
    def is_white(self):
        return False

    def is_blue(self):
        return False
    
    def toogle_mark(self):    
        pass
    
    def is_move_mark(self):
        return False    
    
    def is_marked(self):
        return False       

    def __str__(self):
        return "."
    
class PosMoveField(Field):
    def __init__(self, is_white, window, row, col, board, row_from, col_from, pos_move):
        self.__is_white=is_white
        self.__is_marked =False 
        self.window = window
        self.row = row
        self.col = col
        self.board = board
        self.row_from = row_from
        self.col_from = col_from
        self.pos_move=pos_move
        
            
    def __deepcopy__(self, memo):
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        result.__dict__.update(self.__dict__)
        return result                
        
    def draw(self):
        x = self.col*FIELD_SIZE
        y = self.row*FIELD_SIZE
        pygame.draw.circle(self.window, RED, (x+FIELD_SIZE/2, y+FIELD_SIZE/2), POS_MOVE_MARK_SIZE)
    
    def is_empty(self):
        return True
        
    def is_move_mark(self):
        return True

class Pawn(Field):    
    def __init__(self, is_white, window, row, col, board):
        self.__is_white=is_white
        self.__is_marked =False 
        self.window = window
        self.row = row
        self.col = col
        self.board = board
        
    def __deepcopy__(self, memo):
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        result.__dict__.update(self.__dict__)
        return result  

    def __str__(self):
        if self.is_white():
            return "w"
        return "b"
            
    def is_king(self):
        return False
    
    def is_empty(self):
        return False
    
    def is_white(self):
        return self.__is_white

    def is_blue(self):
        return not self.__is_white
    
    def is_marked(self):
        return self.__is_marked
    
    def toogle_mark(self):
        if self.__is_marked:
            for pos_move in self.pos_moves: #remove possible moves
                row = pos_move.dest_row
                col = pos_move.dest_col
                self.board.board[row][col] = Field()
            self.pos_moves=[]
        else:#self.is_marked
            self.pos_moves = self.board.get_piece_moves(self)
            for pos_move in self.pos_moves:
                row = pos_move.dest_row
                col = pos_move.dest_col
                self.board.board[row][col] = PosMoveField(False, self.window, row, col, self.board, self.row, self.col, pos_move)
            
        self.__is_marked = not self.__is_marked
    
    def draw(self):
        if self.__is_white:
            cur_col = WHITE
        else:
            cur_col = BLUE
        x = self.col*FIELD_SIZE
        y = self.row*FIELD_SIZE
        pygame.draw.circle(self.window, cur_col, (x+FIELD_SIZE/2, y+FIELD_SIZE/2), PIECE_SIZE)

        if self.__is_marked:
            pygame.draw.circle(self.window, RED, (x+FIELD_SIZE/2, y+FIELD_SIZE/2), PIECE_SIZE+MARK_THICK, MARK_THICK)

class King(Pawn):    
    def __init__(self, pawn):
        super().__init__(pawn.is_white(), pawn.window, pawn.row, pawn.col, pawn.board)

    def __deepcopy__(self, memo):
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        result.__dict__.update(self.__dict__)
        return result  
    
    def is_king(self):
        return True

    def __str__(self):
        if self.is_white():
            return "W"
        return "B"
    
    def draw(self):
        if self.is_white():
            cur_col = WHITE
        else:
            cur_col = BLUE
        x = self.col*FIELD_SIZE
        y = self.row*FIELD_SIZE
        pygame.draw.circle(self.window, cur_col, (x+FIELD_SIZE/2, y+FIELD_SIZE/2), PIECE_SIZE)
        pygame.draw.circle(self.window, GREEN, (x+FIELD_SIZE/2, y+FIELD_SIZE/2), PIECE_SIZE/2)

        if self.is_marked():
            pygame.draw.circle(self.window, RED, (x+FIELD_SIZE/2, y+FIELD_SIZE/2), PIECE_SIZE+MARK_THICK, MARK_THICK)

class Board:
    def __init__(self, window): #row, col
        self.board = []#np.full((BOARD_WIDTH, BOARD_WIDTH), None)
        self.window = window
        self.marked_piece = None 
        self.something_is_marked=False
        self.white_turn = True
        self.white_fig_left = 12
        self.blue_fig_left = 12
        self.eval = self.evaluate

        self.__set_pieces()
        
    def __deepcopy__(self, memo):
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        result.__dict__.update(self.__dict__)
        result.board= deepcopy(self.board )
        return result  

    def __str__(self):
        to_ret=""
        for row in range(8):
            for col in range(8):
                to_ret+=str(self.board[row][col])
            to_ret+="\n"
        return to_ret
        
    def __set_pieces(self):
        for row in range(8):
            self.board.append([])
            for col in range(8):
                self.board[row].append( Field() )

        for row in range(3):
            for col in range((row+1) % 2, BOARD_WIDTH, 2):
                self.board[row][col] = Pawn(False, self.window, row, col, self)

        for row in range(5, 8):
            for col in range((row+1) % 2, BOARD_WIDTH, 2):
                self.board[row][col] = Pawn(True, self.window, row, col, self)


    def get_piece_moves(self, piece):
        pos_moves=[]
        row = piece.row
        col = piece.col
        if piece.is_blue():
            enemy_is_white = True
        else:
            enemy_is_white = False

        if piece.is_white() or (piece.is_blue() and piece.is_king()):            
            dir_y = -1
            if row > 0:
                new_row=row+dir_y
                if col > 0:
                    new_col=col-1
                    if self.board[new_row][new_col].is_empty():
                        pos_moves.append(Move(piece, new_row, new_col))  
                        #ruch zwiazany z biciem
                    elif self.board[new_row][new_col].is_white()==enemy_is_white and new_row+dir_y>=0 and new_col-1>=0 and self.board[new_row+dir_y][new_col-1].is_empty():
                        pos_moves.append(Move(piece,new_row+dir_y, new_col-1, self.board[new_row][new_col]))  
                        
                if col < BOARD_WIDTH-1:
                    new_col=col+1
                    if self.board[new_row][new_col].is_empty():
                        pos_moves.append(Move(piece,new_row, new_col))    
                        #ruch zwiazany z biciem
                    elif self.board[new_row][new_col].is_white()==enemy_is_white and new_row+dir_y>=0 and new_col+1<BOARD_WIDTH and self.board[new_row+dir_y][new_col+1].is_empty():
                        pos_moves.append(Move(piece,new_row+dir_y, new_col+1, self.board[new_row][new_col]))  

        if piece.is_blue() or (piece.is_white() and self.board[row][col].is_king()):
            dir_y = 1
            if row<BOARD_WIDTH-1:
                new_row=row+dir_y
                if col > 0:
                    new_col=col-1
                    if self.board[new_row][new_col].is_empty():
                        pos_moves.append(Move(piece,new_row, new_col))    
                    elif self.board[new_row][new_col].is_white()==enemy_is_white and new_row+dir_y<BOARD_WIDTH and new_col-1>=0 and self.board[new_row+dir_y][new_col-1].is_empty():
                        pos_moves.append(Move(piece,new_row+dir_y, new_col-1, self.board[new_row][new_col]))  
                        
                if col < BOARD_WIDTH-1:
                    new_col=col+1
                    if self.board[new_row][new_col].is_empty():
                        pos_moves.append(Move(piece,new_row, new_col))    
                        #ruch zwiazany z biciem
                    elif self.board[new_row][new_col].is_white()==enemy_is_white and new_row+dir_y<BOARD_WIDTH and new_col+1<BOARD_WIDTH and self.board[new_row+dir_y][new_col+1].is_empty():
                        pos_moves.append(Move(piece,new_row+dir_y, new_col+1, self.board[new_row][new_col]))  
        return pos_moves
    
    #ToDo
    def evaluate(self, is_blue_turn):
        h=0
        for row in range(BOARD_WIDTH):
            for col in range((row+1) % 2, BOARD_WIDTH, 2):
                # values for blue turn
                if (self.board[row][col].is_blue()):
                    if (self.board[row][col].is_king()):
                        h -= 10
                    else:
                        h -= 1
                elif (self.board[row][col].is_white()):
                    if (self.board[row][col].is_king()):
                        h += 10
                    else:
                        h += 1
        return h
        if is_blue_turn:
            return h
        else:
            return -h    

    def evaluate2(self, is_blue_turn):
        # numerating from left top
        h=0
        for row in range(BOARD_WIDTH):
            for col in range((row+1) % 2, BOARD_WIDTH, 2):
                # values for blue turn
                if (self.board[row][col].is_blue()):
                    if (self.board[row][col].is_king()):
                        h -= 10
                    else:
                        if col <= 5:
                            h -= 5
                        else:
                            h -= 7
                elif (self.board[row][col].is_white()):
                    if (self.board[row][col].is_king()):
                        h += 10
                    else:
                        if col >= 5:
                            h -= 5
                        else:
                            h -= 7
        return h
                        
    def evaluate3(self, is_blue_turn):
        # numerating from left top
        h=0
        for row in range(BOARD_WIDTH):
            for col in range((row+1) % 2, BOARD_WIDTH, 2):
                # values for blue turn
                if (self.board[row][col].is_blue()):
                    if (self.board[row][col].is_king()):
                        h -= 10
                    else:
                        h -= 1 - col
                elif (self.board[row][col].is_white()):
                    if (self.board[row][col].is_king()):
                        h += 10
                    else:
                        h += 1  + BOARD_WIDTH - col
        return h
    
    def get_possible_moves(self, is_blue_turn):
        pos_moves = []
        for row in range(BOARD_WIDTH):
            for col in range((row+1) % 2, BOARD_WIDTH, 2):
                if not self.board[row][col].is_empty():
                    if (is_blue_turn and self.board[row][col].is_blue()) or (not is_blue_turn and self.board[row][col].is_white()):                        
                        pos_moves.extend( self.get_piece_moves(self.board[row][col]) )
        return pos_moves
        
                
    def draw(self):
        self.window.fill(WHITE)
        for row in range(BOARD_WIDTH):
            for col in range((row+1) % 2, BOARD_WIDTH, 2):
                y = row*FIELD_SIZE
                x = col*FIELD_SIZE
                pygame.draw.rect(self.window, BLACK, (x, y , FIELD_SIZE, FIELD_SIZE))
                self.board[row][col].draw()
                            

  
    def move(self, field):
        d_row = field.row
        d_col = field.col
        row_from = field.row_from
        col_from = field.col_from
        self.board[row_from][col_from].toogle_mark()
        self.something_is_marked = False
        self.board[d_row][d_col]=self.board[row_from][col_from]
        self.board[d_row][d_col].row=d_row
        self.board[d_row][d_col].col=d_col
        self.board[row_from][col_from]=Field()     

        if field.pos_move.captures:
            fig_to_del = field.pos_move.captures
            
            self.board[fig_to_del.row][fig_to_del.col]=Field()
            if self.white_turn:
                self.blue_fig_left -= 1
            else:
                self.white_fig_left -= 1
            
        if self.white_turn and d_row==0:#damka
            self.board[d_row][d_col] = King(self.board[d_row][d_col])

        if not self.white_turn and d_row==BOARD_WIDTH-1:#damka
            self.board[d_row][d_col] = King(self.board[d_row][d_col])
            
        self.white_turn = not self.white_turn
    
    def end(self):
        return self.white_fig_left==0 or self.blue_fig_left==0 or len(self.get_possible_moves(not self.white_turn))==0
        
    def clicked_at(self, row, col):
        field = self.board[row][col]
        if field.is_move_mark():
            self.move(field)
        if (field.is_white() and self.white_turn and not self.something_is_marked) or (field.is_blue() and not self.white_turn and not self.something_is_marked):
            field.toogle_mark()
            self.something_is_marked = True
        elif self.something_is_marked and field.is_marked():
            field.toogle_mark()
            self.something_is_marked = False

    #tu spore powtorzenie kodu z move            
    def make_ai_move(self, move):                
        d_row = move.dest_row
        d_col = move.dest_col
        row_from = move.piece.row
        col_from = move.piece.col

        self.board[d_row][d_col]=self.board[row_from][col_from]
        self.board[d_row][d_col].row=d_row
        self.board[d_row][d_col].col=d_col
        self.board[row_from][col_from]=Field()     

        if move.captures:
            fig_to_del = move.captures
            
            self.board[fig_to_del.row][fig_to_del.col]=Field()
            if self.white_turn:
                self.blue_fig_left -= 1
            else:
                self.white_fig_left -= 1
            
        if self.white_turn and d_row==0:#damka
            self.board[d_row][d_col] = King(self.board[d_row][d_col])

        if not self.white_turn and d_row==BOARD_WIDTH-1:#damka
            self.board[d_row][d_col] = King(self.board[d_row][d_col])
            
        self.white_turn = not self.white_turn
        

class Game:
    def __init__(self, window):
        self.window = window
        self.board = Board(window)
        self.won = 0

    def update(self):
        self.board.draw()
        pygame.display.update()
    
    def mouse_to_indexes(self, pos):
        return (int(pos[0]//FIELD_SIZE), int(pos[1]//FIELD_SIZE))

    def clicked_at(self, pos):
        (col, row) = self.mouse_to_indexes(pos)
        self.board.clicked_at(row, col)
        




def minimax_a_b(board, depth, eval_func):
    if eval_func == 0:
        board.eval = board.evaluate
    elif eval_func == 2:
        board.eval = board.evaluate2
    elif eval_func == 3:
        board.eval = board.evaluate3
    else:
        print("wrong eval_func value") # bieda exception
        raise

    possible_moves = board.get_possible_moves(not board.white_turn)
    values = minimax_a_b_recurr(board, depth, board.white_turn, float('-inf'), float('inf'), 1)

    # print(values)

    if board.white_turn:
        max_value = max(values)
        indices = [index for index, value in enumerate(values) if value == max_value]
        return possible_moves[random.choice(indices)]
    else:
        min_value = min(values)
        indices = [index for index, value in enumerate(values) if value == min_value]
        return possible_moves[random.choice(indices)]

def minimax_a_b_recurr(board, depth, move_max, a, b, first_step):
    initial_board = deepcopy(board)
    curr_board = initial_board
    value = []
    
    if board.end():
        if move_max:
            return 1000
        else:
            return -1000

    if first_step:
        for move in initial_board.get_possible_moves(not initial_board.white_turn):
            curr_board = deepcopy(initial_board)
            curr_board.make_ai_move(move)
            value.append(minimax_a_b_recurr(curr_board, depth -1, curr_board.white_turn, a, b, 0))
        return value

    if depth == 0:
        for move in initial_board.get_possible_moves(not initial_board.white_turn):
            curr_board = deepcopy(initial_board)
            curr_board.make_ai_move(move)
            value.append(curr_board.eval(not curr_board.white_turn))
        if not curr_board.white_turn:
            return max(value)
        else:
            return min(value)
    else:
        if move_max:
            for move in initial_board.get_possible_moves(not initial_board.white_turn):
                curr_board = deepcopy(initial_board)
                curr_board.make_ai_move(move)
                a = max(a, minimax_a_b_recurr(curr_board, depth - 1, curr_board.white_turn, a, b, 0))
                if a >= b:
                    return b
            return a
        else:
            for move in initial_board.get_possible_moves(not initial_board.white_turn):
                curr_board = deepcopy(initial_board)
                curr_board.make_ai_move(move)
                b = min(b, minimax_a_b_recurr(curr_board, depth - 1, curr_board.white_turn, a, b, 0))
                if b <= a:
                    return a
            return b


    # min -> blue       negative evatlution good
    # max -> white      positive evatlution good



def handleWin(white_turn, game):
    if white_turn:
        game.won = 1
    else:
        game.won = -1
    # print("won: ", game.won)


def run_game(max_depth, min_depth, eval_func_max, eval_func_min):
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
    clock = pygame.time.Clock()
    is_running = True
    game = Game(window)
    history = []
    iter = 0

    while is_running and iter < 200:
        clock.tick(FPS)

        # blue AI
        if not game.board.white_turn:
            move = minimax_a_b(deepcopy(game.board), min_depth, eval_func_max)
            game.board.make_ai_move(move)
            history.append(game.board.eval(not game.board.white_turn))
        
        # check win
        if game.board.end():
            is_running = False  
            handleWin(not game.board.white_turn, game)
            break #przydalby sie jakiĹ komunikat kto wygraĹ zamiast break
        
        # white AI
        if game.board.white_turn:
            move = minimax_a_b(deepcopy(game.board), max_depth, eval_func_min)
            game.board.make_ai_move(move)
            history.append(game.board.eval(not game.board.white_turn))

        # check win
        if game.board.end():
            is_running = False  
            handleWin(not game.board.white_turn, game)
            break #przydalby sie jakiĹ komunikat kto wygraĹ zamiast break

        # # white human
        # for event in pygame.event.get():
        #     if event.type == pygame.QUIT:
        #         is_running = False

        #     if event.type == pygame.MOUSEBUTTONDOWN:
        #         pos = pygame.mouse.get_pos()
        #         game.clicked_at(pos)

        iter += 1
        # game.update()

    pygame.quit()

    return history, game.won


%xmode Verbose  

def main():
    max_depth = [1,1,1]
    min_depth = [2,3,4]
    max_eval_func = [0,2,3]
    min_eval_func = [0,2,3]
    # df = pd.DataFrame(columns=['max_depth', 'min_depth', 'max_eval_func','min_eval_func','won'])
    df = pd.read_csv('results' + str(1) + '.csv', index_col=[0])

    # todo
    # steps count if too much same score break
    iter = 0
    for i in max_depth:
        for j in min_depth:
            for k in range(10):
                hist, won = run_game(i, j, max_eval_func[0], min_eval_func[0])  
                df = df.append({'max_depth': i, 'min_depth': j, 'max_eval_func': max_eval_func[0], 'min_eval_func': min_eval_func[0], 'won': won}, ignore_index=True)
                print(f"{iter}: {won}")
                iter += 1
            df.to_csv('results' + str(1) + '.csv')
    
    df.to_csv('results' + str(1) + '.csv')

main()


Exception reporting mode: Verbose
0: 0
0: 0
0: 0
0: 0
0: 0
0: 1
0: 0
0: 0
0: 0
0: -1
0: 0
0: 1
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 1
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: -1
0: 0
0: 0
0: -1
0: -1
0: 0
0: 0
0: 0
0: 0
0: 1
0: 0
0: 0
0: 1
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 1
0: 0
0: 0
0: 0
0: 1
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0
0: 0


In [64]:
df

,max_depth,min_depth,max_eval_func,min_eval_func
0,23,0,0,0
1,23,0,0,0
2,23,0,0,0
3,23,0,0,0
4,23,0,0,0
5,23,0,0,0
6,23,0,0,0
7,23,0,0,0


In [2]:
df.to_csv('test' + datetime.datetime.now() + '.csv')

NameError: name 'df' is not defined

In [40]:

import pandas as pd
df = pd.DataFrame(columns=['max_depth', 'min_depth', 'max_eval_func','min_eval_func'])


In [51]:
df = df.append({'max_depth': 23, 'min_depth': 0, 'max_eval_func': 0, 'min_eval_func': 0}, ignore_index=True)

In [53]:
df.to_csv('test' + str(1) + '.csv')

In [81]:
df = pd.read_csv('results' + str(1) + '.csv', index_col=[0])

In [96]:
df['won'][df['min_depth'] == 1].value_counts()

 0    87
 1     2
-1     1
Name: won, dtype: int64

In [ ]:
# suchy minmax

def minimax_a_b(board, depth):
    initial_board = deepcopy(board)
    # curr_board = initial_board
    # value = []
    # for move in initial_board.get_possible_moves(not initial_board.white_turn):
    #     curr_board = deepcopy(initial_board.make_ai_move(move))
    #     print(move)


    # print(board)
    # return 0
    possible_moves = board.get_possible_moves(not initial_board.white_turn)
    values = minimax_a_b_recurr(board, depth, board.white_turn, 0, 0, 1)

    print(values)

    if board.white_turn:
        return possible_moves[values.index(max(values))]
    else:
        return possible_moves[values.index(min(values))]

def minimax_a_b_recurr(board, depth, move_max, a, b, first_step):
    initial_board = deepcopy(board)
    curr_board = initial_board
    value = []
    depth -= 1

    if depth == 0:
        for move in initial_board.get_possible_moves(not initial_board.white_turn):
            curr_board = deepcopy(initial_board)
            curr_board.make_ai_move(move)
            value.append(curr_board.eval(not curr_board.white_turn))
    else:
        for move in initial_board.get_possible_moves(not initial_board.white_turn):
            curr_board = deepcopy(initial_board)
            curr_board.make_ai_move(move)
            value.append(minimax_a_b_recurr(curr_board, depth, curr_board.white_turn, 0, 0, 0))

    if first_step:
        return value
    
    if move_max:
        return max(value)
    else:
        return min(value)

In [28]:
values = [1,1,4,4,5,6,7,8]

values.index(min(values))

0

In [31]:
max_value = min(values)
indices = [index for index, value in enumerate(values) if value == max_value]
values[random.choice(indices)]

[0, 1]


In [35]:
import random
values[random.choice(indices)]

1

In [42]:
random.choice(indices)

1

In [62]:
%recall 57